In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import isnan
import json
from IPython.display import clear_output

In [2]:
def Load_Data(path_dir):
    list_text = os.listdir(path=path_dir)
    text_array = ''
    for text in list_text:
        with open(path_dir + text, 'r') as f:
            text_array+= f.readline() + '\n'
    text_array = text_array.replace('\ufeff', '')
    list_of_files = text_array.split('\n')
    for i in range(len(list_of_files)):
        list_of_files[i] = list_of_files[i].split('\t')
    text_tmp = []
    for i in range(len(list_of_files)):
        if list_of_files[i] != ['']:
            text_tmp.append(list_of_files[i])
    return text_tmp

In [3]:
def ParseData(data):
    list_of_sites, body_array, values = [],[],[];
    for i in range(len(data)):
        body_array.append([])
        for j in range(len(data[i])):
            if j == 0:
                list_of_sites.append(data[i][j])
            elif j == len(data[i])-1:
                values.append([int(data[i][j])])
            else:
                if j != 4:
                    if data[i][j] == 'null':
                        body_array[i].append(0)
                    elif data[i][j] == 'true':
                        body_array[i].append(1)
                    elif data[i][j] == 'false':
                        if j == 2:
                            body_array[i].append(0)
                        else:
                            body_array[i].append(-1)
                    else:
        #                 print(text_array[i][j])
                        body_array[i].append(float(data[i][j]))
    return np.concatenate((np.array(body_array), np.array(values)), axis=1)

In [4]:
data_all = ParseData(Load_Data('Data/'))
data_test = ParseData(Load_Data('New_test/'))

In [5]:
import torch
import numpy as np

from torch.optim import Adam

from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch import nn

In [15]:
class PhishingRegressor(nn.Module):
    def __init__(self, name=''):
        super().__init__()
        self.name = name
        self.train = None
        self.test = None
        self.epoch = None
        self.loss = nn.BCELoss()
        self.linear1 = nn.Linear(10,10)
        self.linear2 = nn.Linear(10, 1)
#         self.linear2 = nn.Linear(10, 1)
        
    def forward(self, x):
#         x = torch.tanh(self.linear1(x))
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        return x
    
    def Pred(self, data, l):
#         loss_new = nn.BCELoss()
        train_loader = DataLoader(dataset=data, batch_size=l)
        acc_new = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            y_pred = self(X_batch)
            bce_new = self.loss(y_pred, y_batch)
            acc_new = round(sum([int(y_batch[i] == int(y_pred[i][0]>0.5)) for i in range(y_pred.shape[0])])/l, 3)
        return bce_new, acc_new

    def Data_to_Tensor(self, data, length=[200,20,10]):
        ttv = [[],[],[]]
        np.random.shuffle(data)
        j = 0
        num = 0
        classes = [[],[]]
        for i in range(data.shape[0]):
            classes[int(data[i,-1])].append(data[i])
        for j in range(len(classes)):
            num = 0
            u = 0
            for k in range(len(classes[j])):
                if num != length[u]/2:
                    ttv[u].append(classes[j][k])
                    num += 1
                else:
                    if u < len(length) - 1:
                        u += 1
                        num = 0
                    ttv[u].append(classes[j][k])
                    num += 1
        for i in range(len(ttv)):
            ttv[i] = np.array(ttv[i])
            np.random.shuffle(ttv[i])
        out = []
        for i in range(len(ttv)):
            out.append(TensorDataset(torch.tensor(ttv[i][:,:-1], dtype=torch.float), torch.tensor(ttv[i][:,-1], dtype=torch.float).view(-1,1)))
        return out
    def train_model(self, data, length=[200,20,10], f = None):
        optimizer = Adam(self.parameters(), lr=1e-3)
        epochs, bce_train, bce_val= [], [], []
#         loss = nn.BCELoss()
        ltrain, ltest, lval = length
        Data_train, Data_test, Data_val = self.Data_to_Tensor(data)
        train_loader = DataLoader(dataset=Data_train, batch_size=20)
        test_loader = DataLoader(dataset=Data_train, batch_size=ltest)
        train_loader_show = DataLoader(dataset=Data_train, batch_size=ltrain+ltest+lval) 
        epoch = 0
        max_epoch = 5000
        while True:
            epochs.append(epoch)
            for X_batch, y_batch in train_loader: 
                X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
                y_pred = self(X_batch)    
                bce = self.loss(y_pred, y_batch)

                bce.backward()        

                optimizer.step()
                optimizer.zero_grad()       
            
            bce_train.append(bce)
            bce_val.append(self.Pred(Data_val, lval)[0])
            epoch += 1
            if epoch % 100 == 0:
#                 print(bce.detach().numpy().tolist())
                print(f'Epoch = {epoch}, loss = {round(bce.cpu().detach().numpy().tolist(), 5)}, acc_train = {self.Pred(Data_train,ltrain)[1]}, acc_val = {self.Pred(Data_val,lval)[1]}, acc_test = {self.Pred(Data_test,ltest)[1]}')
            if bce <= 0.005:
#                 print(self.Pred(Data_test, ltest))
                print(f'Epoch = {epoch}, loss = {round(bce.cpu().detach().numpy().tolist(), 5)}, acc_train = {self.Pred(Data_train,ltrain)[1]}, acc_val = {self.Pred(Data_val,lval)[1]}, acc_test = {self.Pred(Data_test,ltest)[1]}')
                if f:
                    f.set_title(f'Model №{self.name}')
                    f.plot(epochs,bce_train, color='black')
                    f.plot(epochs,bce_val)
                break
            if epoch == max_epoch:
#                 if f:
#                     f.set_title(f'Model №{self.name}')
#                     f.plot(epochs[:200],bce_train[:200], color='black')
#                     f.plot(epochs[:200],bce_val[:200])
                break
        self.train = train_loader_show
        self.test = test_loader
        self.epoch = epoch 
#         return train_loader_show, test_loader
    def Save_model_data(self):
        for data, val in self.train:
            tmp = []
            for i in range(data.shape[0]):
                dict_tr = {}
                dict_tr['input'] = data[i].detach().numpy().tolist()
                dict_tr['output'] = val[i].detach().numpy().tolist()
                tmp.append(dict_tr)
            with open(f'../js_network/Data/train{self.name}.json','w') as f:
                json.dump(tmp, f)
        for data, val in self.test:
            tmp = []
            for i in range(data.shape[0]):
                dict_tr = {}
                dict_tr['input'] = data[i].detach().numpy().tolist()
                dict_tr['output'] = val[i].detach().numpy().tolist()
                tmp.append(dict_tr)
            with open(f'../js_network/Data/test{self.name}.json','w') as f:
                json.dump(tmp, f)

In [16]:
models = [PhishingRegressor(name=str(i+1)).cuda() for i in range(5)]

In [17]:
%matplotlib
epochs = []
f_mod, axes = plt.subplots(1,len(models), num='NN models')
i = 0
for model in models:
    print(f'Train model №{model.name}:\n')
    model.train_model(data_all, f=axes[i])
    if model.epoch == 5000:
        while model.epoch == 5000:
            model = PhishingRegressor(name=model.name).cuda()
            model.train_model(data_all, f=axes[i])
    epochs.append(model.epoch)
#     model.Save_model_data()
    print()
    i+=1
# with open(f'../js_network/Data/epochs.json','w') as f:
#     json.dump(epochs, f)
epochs

Using matplotlib backend: Qt5Agg
Train model №1:

Epoch = 100, loss = 0.1663, acc_train = 0.92, acc_val = 0.9, acc_test = 0.95
Epoch = 200, loss = 0.08631, acc_train = 0.955, acc_val = 0.9, acc_test = 1.0
Epoch = 300, loss = 0.06554, acc_train = 0.965, acc_val = 0.9, acc_test = 1.0
Epoch = 400, loss = 0.05808, acc_train = 0.965, acc_val = 0.9, acc_test = 1.0
Epoch = 500, loss = 0.0503, acc_train = 0.97, acc_val = 0.9, acc_test = 1.0
Epoch = 600, loss = 0.0432, acc_train = 0.97, acc_val = 0.9, acc_test = 1.0
Epoch = 700, loss = 0.03717, acc_train = 0.97, acc_val = 0.9, acc_test = 1.0
Epoch = 800, loss = 0.03126, acc_train = 0.975, acc_val = 0.9, acc_test = 1.0
Epoch = 900, loss = 0.0264, acc_train = 0.98, acc_val = 0.9, acc_test = 1.0
Epoch = 1000, loss = 0.02222, acc_train = 0.98, acc_val = 0.9, acc_test = 1.0
Epoch = 1100, loss = 0.01843, acc_train = 0.98, acc_val = 0.9, acc_test = 1.0
Epoch = 1200, loss = 0.01539, acc_train = 0.98, acc_val = 0.9, acc_test = 1.0
Epoch = 1300, loss = 0

[2149, 2455, 2803, 4220, 1673]

In [18]:
from sklearn.manifold import TSNE
def get_color_array(array, f=lambda x: x):
    return ['red' if f(array[i]) > 0.5 else 'blue' for i in range(len(array))]

In [19]:
model_to2D = TSNE(learning_rate=1)
transform = model_to2D.fit_transform(data_all[:,:-1])
x = transform[:,0]
y = transform[:,1]

In [20]:
y_val = data_all.T[-1]
y_val_color = get_color_array(y_val, lambda x: x)

In [21]:

data_for_model = TensorDataset(torch.tensor(data_all[:,:-1], dtype=torch.float), torch.tensor(data_all[:,-1], dtype=torch.float).view(-1,1))
data_load = DataLoader(dataset=data_for_model, batch_size=data_all.shape[0])
f_lr, ax = plt.subplots(5,1, num='Res models')
for a,b in data_load:
    a,b = a.cuda(),b.cuda()
    tmp = []
    i = 0
    my_f = lambda x: 0.5
    y_plot = np.array([my_f(x[i]) for i in range(x.shape[0])])
#     print(y_plot)
    for model in models:
        tmp.append(model(a)[:,0].cpu().detach().numpy())
        ax[i].scatter(x,model(a)[:,0].cpu().detach().numpy(), c=y_val_color)
        ax[i].plot(x,y_plot)
        i+=1
    tmp = np.array(tmp).T
    t = b.cpu().detach().numpy()
# Data_con = np.concatenate((tmp.T, t), axis=1)

In [22]:
def res_models(data, models):
    data_for_model = TensorDataset(torch.tensor(data[:,:-1], dtype=torch.float), torch.tensor(data[:,-1], dtype=torch.float).view(-1,1))
    data_load = DataLoader(dataset=data_for_model, batch_size=data.shape[0])
    t = []
    acc_arr = []
    for a,b in data_load:
        a,b = a.cuda(),b.cuda()
        t = b.cpu().detach().numpy()
        tmp = []
        j = 0
        for model in models:
            tmp.append(model(a)[:,0].cpu().detach().numpy())
            acc_arr.append(sum([int(t[i] == int(tmp[j].T[i] >= 0.5)) for i in range(t.shape[0])])/t.shape[0])
            j += 1
    tmp = sum(np.array(tmp)).T/len(models)
    acc = sum([int(t[i] == int(tmp[i] >= 0.5)) for i in range(t.shape[0])])/t.shape[0]
    return [int(tmp[i] >= 0.5) for i in range(t.shape[0])], [acc, acc_arr]

In [23]:
res_all, acc_all = res_models(data_all, models)
acc_all

[1.0,
 [1.0,
  0.9869565217391304,
  0.9869565217391304,
  0.9869565217391304,
  0.9956521739130435]]

In [24]:
res_test, acc_test = res_models(data_test, models)
acc_test

[1.0, [1.0, 0.95, 1.0, 0.95, 1.0]]

In [25]:
max_acc = int(np.array(acc_test[1]).argmax())
max_acc

0

In [30]:
hut = torch.Tensor([1, 0, 1, 1, 0, 1, 1.8936170212765957, 18.333333333333332, 0, 667])
models[0](hut.cuda())

tensor([0.0081], device='cuda:0', grad_fn=<SigmoidBackward>)

In [31]:
for model in models:
    model.Save_model_data()

In [ ]:
with open(f'../js_network/Data/epochs.json','w') as f:
    json.dump(epochs, f)

In [32]:
with open('../js_network/Data/max_acc.json', 'w') as f:
    json.dump(max_acc, f)

In [33]:
res_all_color = get_color_array(res_all)
val_all_color = get_color_array(data_all[:, -1])

In [34]:
%matplotlib
f_train, ax1 = plt.subplots(1,2, sharey=True)
ax1[0].scatter(x, y, c=val_all_color)
ax1[0].set_title('без обработки')
ax1[1].scatter(x, y, c=res_all_color)
ax1[1].set_title('результат модели')
plt.show()

Using matplotlib backend: Qt5Agg


In [35]:
transform_test = model_to2D.fit_transform(data_test)
x_2 = transform_test[:, 0]
y_2 = transform_test[:, 1]

In [36]:
res_test_color = get_color_array(res_test)
val_test_color = get_color_array(data_test[:,-1])
len(res_test_color), len(val_test_color)

(20, 20)

In [37]:
f_test, ax2 = plt.subplots(1,2, sharey=True)
ax2[0].scatter(x_2, y_2, c=val_test_color)
ax2[0].set_title('Тестовое множесто')
ax2[1].scatter(x_2, y_2, c=res_test_color)
ax2[1].set_title('Результат модели')
plt.show()